In [ ]:
!pip install transformers==4.14.1
!pip install bitsandbytes-cuda111
!pip install datasets==1.16.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 59.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 15.1 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=1657dcf0324950126474f39f587073a4c57cfa2e6e82d8a1a8bf5f671dbb2446
  Stored in directory: /root/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 16.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pk

In [ ]:
!git clone https://github.com/feralvam/easse.git

Cloning into 'easse'...
remote: Enumerating objects: 1960, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 1960 (delta 116), reused 102 (delta 102), pack-reused 1819
Receiving objects: 100% (1960/1960), 33.15 MiB | 37.59 MiB/s, done.
Resolving deltas: 100% (1229/1229), done.


In [ ]:
%cd content
%cd easse


[Errno 2] No such file or directory: 'content'
/content
/content/easse


In [ ]:
!ls
!pip install -e .

demo   example.sh  MANIFEST.in	requirements.txt  setup.py
easse  LICENSE	   README.md	setup.cfg	  tests
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/easse
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/text-simplification-evaluation.git (to revision main) to /tmp/pip-install-h9ti640s/tseval_448b946ea964413cbb6dc482990b9b81
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/text-simplification-evaluation.git /tmp/pip-install-h9ti640s/tseval_448b946ea964413cbb6dc482990b9b81
  Resolved https://github.com/facebookresearch/text-simplification-evaluation.git to commit f335e2e27026321c7c3d1dd63857416c7e7397b2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup

In [ ]:
import transformers
from easse.sari import corpus_sari
from easse.bleu import corpus_bleu
import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd
from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise
from tqdm.auto import tqdm
import pandas as pd
from google.colab import files


==============================WARNING: DEPRECATED!==============================
WARNING! This version of bitsandbytes is deprecated. Please switch to `pip install bitsandbytes` and the new repo: https://github.com/TimDettmers/bitsandbytes
==============================WARNING: DEPRECATED!==============================


In [ ]:
from sklearn.model_selection import train_test_split

import transformers

import pandas as pd

import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd

from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise

from tqdm.auto import tqdm
import transformers


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
class FrozenBNBLinear(nn.Module):
    def __init__(self, weight, absmax, code, bias=None):
        assert isinstance(bias, nn.Parameter) or bias is None
        super().__init__()
        self.out_features, self.in_features = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
        self.bias = bias
 
    def forward(self, input):
        output = DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias)
        if self.adapter:
            output += self.adapter(input)
        return output
 
    @classmethod
    def from_linear(cls, linear: nn.Linear) -> "FrozenBNBLinear":
        weights_int8, state = quantize_blockise_lowmemory(linear.weight)
        return cls(weights_int8, *state, linear.bias)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.in_features}, {self.out_features})"
 
class DequantizeAndLinear(torch.autograd.Function): 
  @staticmethod
  @custom_fwd
  def forward(ctx, input: torch.Tensor, weights_quantized: torch.ByteTensor,
              absmax: torch.FloatTensor, code: torch.FloatTensor, bias: torch.FloatTensor):
      weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
      ctx.save_for_backward(input, weights_quantized, absmax, code)
      ctx._has_bias = bias is not None
      return F.linear(input, weights_deq, bias).clone()
 
  @staticmethod
  @custom_bwd
  def backward(ctx, grad_output: torch.Tensor):
      assert not ctx.needs_input_grad[1] and not ctx.needs_input_grad[2] and not ctx.needs_input_grad[3]
      input, weights_quantized, absmax, code = ctx.saved_tensors
      # grad_output: [*batch, out_features]
      weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
      grad_input = grad_output @ weights_deq
      grad_bias = grad_output.flatten(0, -2).sum(dim=0) if ctx._has_bias else None
      return grad_input, None, None, None, grad_bias

class FrozenBNBEmbedding(nn.Module):
    def __init__(self, weight, absmax, code):
        super().__init__()
        self.num_embeddings, self.embedding_dim = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
 
    def forward(self, input, **kwargs):
        with torch.no_grad():
            # note: both quantuized weights and input indices are *not* differentiable
            weight_deq = dequantize_blockwise(self.weight, absmax=self.absmax, code=self.code)
            output = F.embedding(input, weight_deq, **kwargs)
        if self.adapter:
            output += self.adapter(input)
        return output 

    @classmethod
    def from_embedding(cls, embedding: nn.Embedding) -> "FrozenBNBEmbedding":
        weights_int8, state = quantize_blockise_lowmemory(embedding.weight)
        return cls(weights_int8, *state)
 
    def __repr__(self):
      return f"{self.__class__.__name__}({self.num_embeddings}, {self.embedding_dim})"



def quantize_blockise_lowmemory(matrix: torch.Tensor, chunk_size: int = 2 ** 20):
    assert chunk_size % 4096 == 0
    code = None
    chunks = []
    absmaxes = []
    flat_tensor = matrix.view(-1)
    for i in range((matrix.numel() - 1) // chunk_size + 1):
        input_chunk = flat_tensor[i * chunk_size: (i + 1) * chunk_size].clone()
        quantized_chunk, (absmax_chunk, code) = quantize_blockwise(input_chunk, code=code)
        chunks.append(quantized_chunk)
        absmaxes.append(absmax_chunk)
 
    matrix_i8 = torch.cat(chunks).reshape_as(matrix)
    absmax = torch.cat(absmaxes)
    return matrix_i8, (absmax, code)



def convert_to_int8(model):
    """Convert linear and embedding modules to 8-bit with optional adapters"""
    for module in list(model.modules()):
        for name, child in module.named_children():
            if isinstance(child, nn.Linear):
                print(name, child)
                setattr(
                    module,
                    name,
                    FrozenBNBLinear(
                        weight=torch.zeros(child.out_features, child.in_features, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                        bias=child.bias,
                    ),
                )
            elif isinstance(child, nn.Embedding):
                setattr(
                    module,
                    name,
                    FrozenBNBEmbedding(
                        weight=torch.zeros(child.num_embeddings, child.embedding_dim, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                    )
                )
 

In [ ]:
class GPTJBlock(transformers.models.gptj.modeling_gptj.GPTJBlock):
    def __init__(self, config):
        super().__init__(config)

        convert_to_int8(self.attn)
        convert_to_int8(self.mlp)


class GPTJModel(transformers.models.gptj.modeling_gptj.GPTJModel):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)
        

class GPTJForCausalLM(transformers.models.gptj.modeling_gptj.GPTJForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)


transformers.models.gptj.modeling_gptj.GPTJBlock = GPTJBlock

In [ ]:
class T5ForConditionalGeneration(transformers.models.t5.modeling_t5.T5ForConditionalGeneration):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)

transformers.models.t5.modeling_t5.T5ForConditionalGeneration = T5ForConditionalGeneration

In [ ]:
config = transformers.GPTJConfig.from_pretrained("EleutherAI/gpt-j-6B")
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

Downloading:   0%|          | 0.00/930 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.94k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

In [ ]:
config.pad_token_id = config.eos_token_id
tokenizer.pad_token = config.pad_token_id

In [ ]:
gpt = GPTJForCausalLM.from_pretrained("hivemind/gpt-j-6B-8bit", low_cpu_mem_usage=True)
#gpt = GPTJForCausalLM.from_pretrained("gustavecortal/fr-boris-8bit", low_cpu_mem_usage=True) French GPT-J Cedille's Boris

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.75G [00:00<?, ?B/s]

k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, 

In [ ]:
def add_adapters(model, adapter_dim=4, p = 0.1):
    assert adapter_dim > 0

    for name, module in model.named_modules():
      if isinstance(module, FrozenBNBLinear):
          if "attn" in name or "mlp" in name or "head" in name:
              print("Adding adapter to", name)
              module.adapter = nn.Sequential(
                nn.Linear(module.in_features, adapter_dim, bias=False),
                nn.Dropout(p=p),
                nn.Linear(adapter_dim, module.out_features, bias=False),
            )
              print("Initializing", name)
              nn.init.zeros_(module.adapter[2].weight)

          else:
              print("Not adding adapter to", name)
      elif isinstance(module, FrozenBNBEmbedding):
          print("Adding adapter to", name)
          module.adapter = nn.Sequential(
                nn.Embedding(module.num_embeddings, adapter_dim),
                nn.Dropout(p=p),
                nn.Linear(adapter_dim, module.embedding_dim, bias=False),
            )
          print("Initializing", name)
          nn.init.zeros_(module.adapter[2].weight)

add_adapters(gpt)
device =  'cpu'
gpt.to(device)

Adding adapter to transformer.wte
Initializing transformer.wte
Adding adapter to transformer.h.0.attn.k_proj
Initializing transformer.h.0.attn.k_proj
Adding adapter to transformer.h.0.attn.v_proj
Initializing transformer.h.0.attn.v_proj
Adding adapter to transformer.h.0.attn.q_proj
Initializing transformer.h.0.attn.q_proj
Adding adapter to transformer.h.0.attn.out_proj
Initializing transformer.h.0.attn.out_proj
Adding adapter to transformer.h.0.mlp.fc_in
Initializing transformer.h.0.mlp.fc_in
Adding adapter to transformer.h.0.mlp.fc_out
Initializing transformer.h.0.mlp.fc_out
Adding adapter to transformer.h.1.attn.k_proj
Initializing transformer.h.1.attn.k_proj
Adding adapter to transformer.h.1.attn.v_proj
Initializing transformer.h.1.attn.v_proj
Adding adapter to transformer.h.1.attn.q_proj
Initializing transformer.h.1.attn.q_proj
Adding adapter to transformer.h.1.attn.out_proj
Initializing transformer.h.1.attn.out_proj
Adding adapter to transformer.h.1.mlp.fc_in
Initializing transfor

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): FrozenBNBEmbedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): FrozenBNBLinear(4096, 4096)
          (v_proj): FrozenBNBLinear(4096, 4096)
          (q_proj): FrozenBNBLinear(4096, 4096)
          (out_proj): FrozenBNBLinear(4096, 4096)
        )
        (mlp): GPTJMLP(
          (fc_in): FrozenBNBLinear(4096, 16384)
          (fc_out): FrozenBNBLinear(16384, 4096)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0

In [ ]:
%cd ..

/content


In [ ]:
gpt.load_state_dict(torch.load('gdrive/MyDrive/Final Project Data/model_tensor_all_lora_with_newsela_new.pt'))

<All keys matched successfully>

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
gpt.to(device)

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): FrozenBNBEmbedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): FrozenBNBLinear(4096, 4096)
          (v_proj): FrozenBNBLinear(4096, 4096)
          (q_proj): FrozenBNBLinear(4096, 4096)
          (out_proj): FrozenBNBLinear(4096, 4096)
        )
        (mlp): GPTJMLP(
          (fc_in): FrozenBNBLinear(4096, 16384)
          (fc_out): FrozenBNBLinear(16384, 4096)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0

#run model from advance to intermediate on train set one_stop_english


> Indented block



In [ ]:
df_a_t_i_one_stop_english = pd.read_csv(r'ADV_INT_train_one_stop_english.csv')

FileNotFoundError: ignored

In [ ]:
orig_sents_a_t_t_os = []
sys_sents_a_t_i_os = []
for i in df_a_t_i_one_stop_english['text'][0:100]:
  orig_sents_a_t_t_os.append(i.split('text:')[1].split('to:')[0])
  prompt = tokenizer(i, return_tensors='pt')
  prompt = {key: value.to(device) for key, value in prompt.items()}
  out = gpt.generate(**prompt, min_length=128, max_length=128, do_sample=True)
  sys_sents_a_t_i_os.append(tokenizer.decode(out[0]).split('text:')[1].split('to:')[1])

  


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 135, but ``max_length`` is set to 1

In [ ]:
corpus_sari(orig_sents = orig_sents_a_t_t_os,  
            sys_sents = sys_sents_a_t_i_os, 
            refs_sents= [orig_sents_a_t_t_os] )

29.375672501228024

In [ ]:
corpus_bleu(sys_sents = sys_sents_a_t_i_os, 
            refs_sents=[orig_sents_a_t_t_os])

37.84269963070405

#run model from intermediate to elementary on train set one_stop_english

In [ ]:
df_i_t_e_one_stop_english = pd.read_csv(r'INT_ELE_train_one_stop_english.csv')

In [ ]:
orig_sents_i_t_e_os = []
sys_sents_i_t_e_os = []
for i in df_i_t_e_one_stop_english['text'][0:100]:
  orig_sents_i_t_e_os.append(i.split('text:')[1].split('to:')[0])
  prompt = tokenizer(i, return_tensors='pt')
  prompt = {key: value.to(device) for key, value in prompt.items()}
  out = gpt.generate(**prompt, min_length=128, max_length=128, do_sample=True)
  sys_sents_i_t_e_os.append(tokenizer.decode(out[0]).split('text:')[1].split('to:')[1])

  


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
corpus_sari(orig_sents = orig_sents_i_t_e_os,  
            sys_sents = sys_sents_i_t_e_os, 
            refs_sents= [orig_sents_i_t_e_os] )

28.089007635778515

In [ ]:
corpus_bleu(sys_sents = sys_sents_i_t_e_os, 
            refs_sents=[orig_sents_i_t_e_os])

35.43093634107986

#run model from advance to intermediate on train set new_sela


In [ ]:
df_a_t_i_train_new_sela = pd.read_csv(r'df_newSela_fine_tune_ad_int_train.csv')

In [ ]:
orig_sents_a_t_i_ns = []
sys_sents_a_t_i_ns = []
for i in df_a_t_i_train_new_sela['text'][0:100]:
  orig_sents_a_t_i_ns.append(i.split('text:')[1].split('to:')[0])
  prompt = tokenizer(i, return_tensors='pt')
  prompt = {key: value.to(device) for key, value in prompt.items()}
  out = gpt.generate(**prompt, min_length=128, max_length=128, do_sample=True)
  sys_sents_a_t_i_ns.append(tokenizer.decode(out[0]).split('text:')[1].split('to:')[1])

  


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
corpus_sari(orig_sents = orig_sents_a_t_i_ns,  
            sys_sents = sys_sents_a_t_i_ns, 
            refs_sents= [orig_sents_a_t_i_ns] )

30.540529468920155

In [ ]:
corpus_bleu(sys_sents = sys_sents_a_t_i_ns, 
            refs_sents=[orig_sents_a_t_i_ns])

49.399305336014855

#run model from intermediate to elementary on train set new_sela

1.   List item
2.   List item



In [ ]:
df_i_t_e_train_new_sela = pd.read_csv(r'df_newSela_fine_tune_int_el_train.csv')

In [ ]:
orig_sents_i_t_e_ns = []
sys_sents_i_t_e_ns = []
for i in df_i_t_e_train_new_sela['text'][0:100]:
  orig_sents_i_t_e_ns.append(i.split('text:')[1].split('to:')[0])
  prompt = tokenizer(i, return_tensors='pt')
  prompt = {key: value.to(device) for key, value in prompt.items()}
  out = gpt.generate(**prompt, min_length=128, max_length=128, do_sample=True)
  sys_sents_i_t_e_ns.append(tokenizer.decode(out[0]).split('text:')[1].split('to:')[1])

  


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
corpus_sari(orig_sents = orig_sents_i_t_e_ns,  
            sys_sents = sys_sents_i_t_e_ns, 
            refs_sents= [orig_sents_i_t_e_ns] )

30.29608380730262

In [ ]:
corpus_bleu(sys_sents = sys_sents_i_t_e_ns, 
            refs_sents=[orig_sents_i_t_e_ns])

40.98485736508661

#run model from intermediate to elementary on train set new_sela
## New Section

In [ ]:
df_wiki_train = pd.read_csv(r'Wiki_train.csv')

In [ ]:
orig_sents_wiki = []
sys_sents_wiki = []
for i in df_wiki_train['text'][0:99]:
  orig_sents_wiki.append(i.split('text:')[2].split('to:')[0])
  prompt = tokenizer(i, return_tensors='pt')
  prompt = {key: value.to(device) for key, value in prompt.items()}
  out = gpt.generate(**prompt, min_length=128, max_length=128, do_sample=True)

  sys_sents_wiki.append(tokenizer.decode(out[0]).split('text:')[2].split('to:')[1])

  


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
corpus_sari(orig_sents = [orig_sents_wiki[0]],  
            sys_sents = [sys_sents_wiki[0]], 
            refs_sents= [[orig_sents_wiki[0]]] )

30.43028451433624

In [ ]:
corpus_bleu(sys_sents = [sys_sents_wiki[0]], 
            refs_sents=[[orig_sents_wiki[0]]])

84.28014430784187

In [ ]:
sys_sents_wiki

[' Hofstetten-Fl Ö³Â¼h is a municipality in the district Dorneck in the canton of Solothurn in Switzerland. \n ',
 " It quickly strengthened into Tropical Storm Marco with 65 mph winds that afternoon. \n floridabrewer102 said : Even after the storm made a turn toward the east, it's strength continued to grow.\n ",
 ' Thomas Eastoe Abbott was an English poet  \n}}\n\nBut she found a way to keep herself busy.\n\nBut she found a way to keep herself busy.\n\nBut sheFound a way to keep herself busy.\n\nbut she found a way to keep herself busy.\n\nBut she found a way to keep herself busy.\n\nbut she found a way to keep herself busy.\n\nBut she found a way to keep herself busy.\n\nbut she found a way to hide',
 ' The SAT Reasoning Test is a test for college admissions in the United States. \n v: The SAT Reasoning Test Is a standardized test for college admissions in the United States.\n\n\' The SAT is one of the most important steps in getting into a school,\'said Joe Tsoulas, a director of c

In [ ]:
sari_l = []
belu_l = []
for i in range(len(orig_sents_wiki)):
    sari_l.append(corpus_sari(orig_sents = [orig_sents_wiki[i]],  
            sys_sents = [sys_sents_wiki[i]], 
            refs_sents= [[orig_sents_wiki[i]]]))
    belu_l.append(corpus_bleu(sys_sents = [sys_sents_wiki[i]], 
            refs_sents=[[orig_sents_wiki[i]]]))
df = pd.DataFrame()
df['original'] = orig_sents_wiki
df['simplified'] = sys_sents_wiki
df['sari'] = sari_l
df['belu'] = belu_l
df.to_csv('results.csv')

In [ ]:
df = pd.DataFrame()

In [ ]:
df = pd.DataFrame()
df['original'] = orig_sents_wiki
df['simplified'] = sys_sents_wiki
df['sari'] = sari_l
df['belu'] = belu_l
df.to_csv('results.csv')

In [ ]:
df.to_csv('results.csv')